In [13]:
import pandas as pd

train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
train_df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [14]:
title_rev = train_df[['title','revenue']]
title_rev.head()

,title,revenue
0,Hot Tub Time Machine 2,12314651
1,The Princess Diaries 2: Royal Engagement,95149435
2,Whiplash,13092000
3,Kahaani,16000000
4,Marine Boy,3923970


In [15]:
#create_dictionary
dictionary = set()
def make_dict(x):
    global dictionary
    x = x.split(" ")
    for i in x:
        dictionary.add(i)

title_rev['title'].apply(make_dict)

def set_to_dict(s):
    s = list(s)
    dictionary = dict()
    for i in range(len(s)):
        dictionary[(s[i])] = i
    return dictionary
dictionary = set_to_dict(dictionary)

In [16]:
import numpy as np
word_vec = []
def create_word_vec(x):
    global word_vec
    global dictionary
    x = x.split(" ")
    vec = [0.0]*50
    for i in range(len(x)):
        if x[i] in dictionary:
            vec[i]=dictionary[x[i]]
    word_vec.append(vec)

title_rev['title'].apply(create_word_vec)
word_vec = np.array(word_vec)

In [17]:
df = pd.DataFrame({"title":list(word_vec)})
df['revenue']  = title_rev['revenue']

In [18]:
from keras import models, layers, regularizers, optimizers
def create_embedding_model():
    model = models.Sequential()
    model.add(layers.Embedding(len(dictionary),30, input_length = 50));
    model.add(layers.Flatten())
    # model.add(layers.Dense(256,activation = 'relu'))
    # model.add(layers.Dropout(0.7))
    model.add(layers.Dense(128,activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64,activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam',loss='mean_squared_logarithmic_error',metrics=['mean_squared_logarithmic_error'])
    return model
def perform_k_fold(model_func, data, y_label, epochs, batch_size,x_label = '', k = 10,IsSeries = False):
    data = data.sample(frac=1)
    train_x = data.drop(y_label,axis = 1)
    train_y = data[y_label]
    split_value = int(data.shape[0]/k)
    split_index = 0
    metric_list = []
    loss_list = []
    for i in range(k):
        valid_x = train_x.iloc[split_index:split_value+split_index]
        valid_y = train_y.iloc[split_index:split_value+split_index]
        tr_x = train_x.drop(valid_x.index)
        tr_y = train_y.drop(valid_y.index)
        if IsSeries:
            valid_x = valid_x[x_label]
            tr_x = tr_x[x_label]
            valid_x = np.stack(valid_x.to_numpy(),axis = 0)
            tr_x = np.stack(tr_x.to_numpy(),axis = 0)
       
        model = model_func()
        model.fit(tr_x,tr_y,epochs=epochs,batch_size = batch_size)
        loss, metric = model.evaluate(tr_x,tr_y)
        metric_list.append(metric)
        loss_list.append(loss)
    print("Average Loss: ",sum(loss_list)/len(loss_list)," Average metric",sum(metric_list)/len(metric_list))


In [66]:
perform_k_fold(create_embedding_model, df, 'revenue',134,32, IsSeries = True, x_label = 'title')

Epoch 1/134
2700/2700 [==============================] - 1s 467us/step - loss: 170.4246 - mean_squared_logarithmic_error: 170.4246
Epoch 2/134
2700/2700 [==============================] - 0s 99us/step - loss: 103.1686 - mean_squared_logarithmic_error: 103.1686
Epoch 3/134
2700/2700 [==============================] - 0s 99us/step - loss: 80.3570 - mean_squared_logarithmic_error: 80.3570
Epoch 4/134
2700/2700 [==============================] - 0s 97us/step - loss: 67.3177 - mean_squared_logarithmic_error: 67.3177
Epoch 5/134
2700/2700 [==============================] - 0s 97us/step - loss: 58.5092 - mean_squared_logarithmic_error: 58.5092
Epoch 6/134
2700/2700 [==============================] - 0s 96us/step - loss: 51.9100 - mean_squared_logarithmic_error: 51.9100
Epoch 7/134
2700/2700 [==============================] - 0s 98us/step - loss: 47.0213 - mean_squared_logarithmic_error: 47.0213
Epoch 8/134
2700/2700 [==============================] - 0s 99us/step - loss: 42.9246 - mean_square

2700/2700 [==============================] - 0s 98us/step - loss: 9.2218 - mean_squared_logarithmic_error: 9.2218
Epoch 129/134
2700/2700 [==============================] - 0s 105us/step - loss: 9.3398 - mean_squared_logarithmic_error: 9.3398
Epoch 130/134
2700/2700 [==============================] - 0s 99us/step - loss: 9.2327 - mean_squared_logarithmic_error: 9.2327
Epoch 131/134
2700/2700 [==============================] - 0s 104us/step - loss: 9.2395 - mean_squared_logarithmic_error: 9.2395
Epoch 132/134
2700/2700 [==============================] - 0s 101us/step - loss: 9.1877 - mean_squared_logarithmic_error: 9.1877
Epoch 133/134
2700/2700 [==============================] - 0s 106us/step - loss: 9.1848 - mean_squared_logarithmic_error: 9.1848
Epoch 134/134
2700/2700 [==============================] - 0s 146us/step
Epoch 1/134
2700/2700 [==============================] - 1s 479us/step - loss: 173.4099 - mean_squared_logarithmic_error: 173.4099
Epoch 2/134
2700/2700 [===============

2700/2700 [==============================] - 0s 97us/step - loss: 9.4488 - mean_squared_logarithmic_error: 9.4488
Epoch 121/134
2700/2700 [==============================] - 0s 99us/step - loss: 9.4341 - mean_squared_logarithmic_error: 9.4341
Epoch 122/134
2700/2700 [==============================] - 0s 101us/step - loss: 9.3254 - mean_squared_logarithmic_error: 9.3254
Epoch 123/134
2700/2700 [==============================] - 0s 102us/step - loss: 9.3770 - mean_squared_logarithmic_error: 9.3770
Epoch 124/134
2700/2700 [==============================] - 0s 100us/step - loss: 9.3908 - mean_squared_logarithmic_error: 9.3908
Epoch 125/134
2700/2700 [==============================] - 0s 101us/step - loss: 9.3674 - mean_squared_logarithmic_error: 9.3674
Epoch 126/134
2700/2700 [==============================] - 0s 101us/step - loss: 9.4396 - mean_squared_logarithmic_error: 9.4396
Epoch 127/134
2700/2700 [==============================] - 0s 100us/step - loss: 9.3747 - mean_squared_logarithmi

Epoch 49/134
2700/2700 [==============================] - 0s 102us/step - loss: 10.7488 - mean_squared_logarithmic_error: 10.7488
Epoch 50/134
2700/2700 [==============================] - 0s 102us/step - loss: 10.6407 - mean_squared_logarithmic_error: 10.6407
Epoch 51/134
2700/2700 [==============================] - 0s 101us/step - loss: 10.6334 - mean_squared_logarithmic_error: 10.6334
Epoch 52/134
2700/2700 [==============================] - 0s 100us/step - loss: 10.5375 - mean_squared_logarithmic_error: 10.5375
Epoch 53/134
2700/2700 [==============================] - 0s 101us/step - loss: 10.5404 - mean_squared_logarithmic_error: 10.5404
Epoch 54/134
2700/2700 [==============================] - 0s 100us/step - loss: 10.4243 - mean_squared_logarithmic_error: 10.4243
Epoch 55/134
2700/2700 [==============================] - 0s 100us/step - loss: 10.3948 - mean_squared_logarithmic_error: 10.3948
Epoch 56/134
2700/2700 [==============================] - 0s 101us/step - loss: 10.3228 - 

2700/2700 [==============================] - 0s 97us/step - loss: 12.8619 - mean_squared_logarithmic_error: 12.8619
Epoch 42/134
2700/2700 [==============================] - 0s 97us/step - loss: 12.5491 - mean_squared_logarithmic_error: 12.5491
Epoch 43/134
2700/2700 [==============================] - 0s 97us/step - loss: 12.2904 - mean_squared_logarithmic_error: 12.2904
Epoch 44/134
2700/2700 [==============================] - 0s 97us/step - loss: 12.2326 - mean_squared_logarithmic_error: 12.2326
Epoch 45/134
2700/2700 [==============================] - 0s 96us/step - loss: 12.0139 - mean_squared_logarithmic_error: 12.0139
Epoch 46/134
2700/2700 [==============================] - 0s 96us/step - loss: 11.9769 - mean_squared_logarithmic_error: 11.9769
Epoch 47/134
2700/2700 [==============================] - 0s 97us/step - loss: 11.6526 - mean_squared_logarithmic_error: 11.6526
Epoch 48/134
2700/2700 [==============================] - 0s 97us/step - loss: 11.5114 - mean_squared_logarith

...

2700/2700 [==============================] - 0s 101us/step - loss: 9.6104 - mean_squared_logarithmic_error: 9.6104
Epoch 90/134
2700/2700 [==============================] - 0s 99us/step - loss: 9.6084 - mean_squared_logarithmic_error: 9.6084
Epoch 91/134
2700/2700 [==============================] - 0s 100us/step - loss: 9.5527 - mean_squared_logarithmic_error: 9.5527
Epoch 92/134
2700/2700 [==============================] - 0s 99us/step - loss: 9.5445 - mean_squared_logarithmic_error: 9.5445
Epoch 93/134
2700/2700 [==============================] - 0s 101us/step - loss: 9.5707 - mean_squared_logarithmic_error: 9.5707
Epoch 94/134
2700/2700 [==============================] - 0s 99us/step - loss: 9.6095 - mean_squared_logarithmic_error: 9.6095
Epoch 95/134
2700/2700 [==============================] - 0s 101us/step - loss: 9.5902 - mean_squared_logarithmic_error: 9.5902
Epoch 96/134
2700/2700 [==============================] - 0s 100us/step - loss: 9.5959 - mean_squared_logarithmic_error:

2700/2700 [==============================] - 0s 132us/step - loss: 9.6765 - mean_squared_logarithmic_error: 9.6765
Epoch 82/134
2700/2700 [==============================] - 0s 123us/step - loss: 9.6594 - mean_squared_logarithmic_error: 9.6594
Epoch 83/134
2700/2700 [==============================] - 0s 107us/step - loss: 9.6870 - mean_squared_logarithmic_error: 9.6870
Epoch 84/134
2700/2700 [==============================] - 0s 116us/step - loss: 9.6562 - mean_squared_logarithmic_error: 9.6562
Epoch 85/134
2700/2700 [==============================] - 0s 112us/step - loss: 9.5964 - mean_squared_logarithmic_error: 9.5964
Epoch 86/134
2700/2700 [==============================] - 0s 115us/step - loss: 9.6238 - mean_squared_logarithmic_error: 9.6238
Epoch 87/134
2700/2700 [==============================] - 0s 109us/step - loss: 9.5726 - mean_squared_logarithmic_error: 9.5726
Epoch 88/134
2700/2700 [==============================] - 0s 109us/step - loss: 9.6020 - mean_squared_logarithmic_err

2700/2700 [==============================] - 0s 103us/step - loss: 9.7208 - mean_squared_logarithmic_error: 9.7208
Epoch 74/134
2700/2700 [==============================] - 0s 110us/step - loss: 9.7332 - mean_squared_logarithmic_error: 9.7332
Epoch 75/134
2700/2700 [==============================] - 0s 106us/step - loss: 9.6615 - mean_squared_logarithmic_error: 9.6615
Epoch 76/134
2700/2700 [==============================] - 0s 106us/step - loss: 9.6619 - mean_squared_logarithmic_error: 9.6619
Epoch 77/134
2700/2700 [==============================] - 0s 106us/step - loss: 9.6094 - mean_squared_logarithmic_error: 9.6094
Epoch 78/134
2700/2700 [==============================] - 0s 109us/step - loss: 9.6911 - mean_squared_logarithmic_error: 9.6911
Epoch 79/134
2700/2700 [==============================] - 0s 109us/step - loss: 9.5889 - mean_squared_logarithmic_error: 9.5889
Epoch 80/134
2700/2700 [==============================] - 0s 107us/step - loss: 9.5923 - mean_squared_logarithmic_err

2700/2700 [==============================] - 0s 103us/step - loss: 9.8161 - mean_squared_logarithmic_error: 9.8161
Epoch 66/134
2700/2700 [==============================] - 0s 103us/step - loss: 9.8638 - mean_squared_logarithmic_error: 9.8638
Epoch 67/134
2700/2700 [==============================] - 0s 103us/step - loss: 9.8834 - mean_squared_logarithmic_error: 9.8834
Epoch 68/134
2700/2700 [==============================] - 0s 103us/step - loss: 9.7974 - mean_squared_logarithmic_error: 9.7974
Epoch 69/134
2700/2700 [==============================] - 0s 102us/step - loss: 9.7754 - mean_squared_logarithmic_error: 9.7754
Epoch 70/134
2700/2700 [==============================] - 0s 104us/step - loss: 9.7272 - mean_squared_logarithmic_error: 9.7272
Epoch 71/134
2700/2700 [==============================] - 0s 103us/step - loss: 9.7686 - mean_squared_logarithmic_error: 9.7686
Epoch 72/134
2700/2700 [==============================] - 0s 103us/step - loss: 9.6726 - mean_squared_logarithmic_err

2700/2700 [==============================] - 0s 104us/step - loss: 10.9116 - mean_squared_logarithmic_error: 10.9116
Epoch 57/134
2700/2700 [==============================] - 0s 104us/step - loss: 10.8250 - mean_squared_logarithmic_error: 10.8250
Epoch 58/134
2700/2700 [==============================] - 0s 105us/step - loss: 10.7167 - mean_squared_logarithmic_error: 10.7167
Epoch 59/134
2700/2700 [==============================] - 0s 102us/step - loss: 10.7025 - mean_squared_logarithmic_error: 10.7025
Epoch 60/134
2700/2700 [==============================] - 0s 103us/step - loss: 10.5564 - mean_squared_logarithmic_error: 10.5564
Epoch 61/134
2700/2700 [==============================] - 0s 103us/step - loss: 10.5208 - mean_squared_logarithmic_error: 10.5208
Epoch 62/134
2700/2700 [==============================] - 0s 105us/step - loss: 10.4327 - mean_squared_logarithmic_error: 10.4327
Epoch 63/134
2700/2700 [==============================] - 0s 104us/step - loss: 10.3974 - mean_squared_

In [19]:
def model1():
    model1 = models.Sequential()
    model1.add(layers.Dense(356, activation='relu', input_shape=(3,)))
    model1.add(layers.Dropout(0.1))
    model1.add(layers.Dense(256, activation='relu'))
    model1.add(layers.Dense(10,activation='relu'))
    model1.add(layers.Dense(1))
    
    model1.compile(optimizer=optimizers.rmsprop(lr=1e-5),loss='mse',metrics=['mean_squared_logarithmic_error'])
    return model1
df = train_df[['revenue','budget','runtime','popularity']]
df = df.fillna(0)

df = df.apply(np.log)
def f(x):
    if x<0:
        return 0
    else:
        return x
df = df.applymap(f)



In [ ]:
perform_k_fold(model1, df, 'revenue', epochs = 40, batch_size = 32)

In [20]:
def model2(): 
    model2 = models.Sequential()
    model2.add(layers.Dense(356, activation='relu', input_shape=(2,)))
    model2.add(layers.Dropout(0.2))
    model2.add(layers.BatchNormalization())
    model2.add(layers.Dense(256, activation='relu'))
    model2.add(layers.Dropout(0.2))
    model2.add(layers.Dense(128, activation='relu'))
    model2.add(layers.Dropout(0.2))
    model2.add(layers.Dense(10,activation='relu'))
    model2.add(layers.Dense(1))

    model2.compile(optimizer=optimizers.rmsprop(lr=1e-5),loss='mse',metrics=['mean_squared_logarithmic_error'])
    return model2
df = train_df[['revenue','crew','cast']]
df['revenue'] = df['revenue'].apply(np.log)
df['revenue'] = df['revenue'].apply(f)
df['crew'] = df['crew'].apply(lambda x: x.count('{') if type(x)==str else 0)
df['cast'] = df['cast'].apply(lambda x: x.count('{') if type(x)==str else 0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [ ]:
perform_k_fold(model2, df, 'revenue', epochs = 50, batch_size = 32)

In [23]:
def model3():
    model3 = models.Sequential()
    model3.add(layers.Dense(356, activation='relu', input_shape=(2,)))
    model3.add(layers.Dropout(0.2))
    model3.add(layers.Dense(256, activation='relu'))
    model3.add(layers.Dense(10,activation='relu'))
    model3.add(layers.Dense(1))
    model3.compile(optimizer=optimizers.rmsprop(lr=1e-5),loss='mse',metrics=['mean_squared_logarithmic_error'])
    return model3
def ap(x):
    #Convert years (ex: 86 -> 1986, 15 -> 2015)
    if type(x) == str:
        x = int(x.split('/')[2])
        if x<=19:
            return 2000+x
        else:
            return 1900+x
    else:
        return 0
df = pd.DataFrame({"Month":train_df['release_date'].apply(lambda x: int(x.split('/')[0]) if type(x)==str else 0 ),'Year':train_df['release_date'].apply(ap),'revenue':train_df['revenue'].apply(np.log).apply(f)})

In [ ]:
perform_k_fold(model3, df, 'revenue', epochs = 50, batch_size = 32)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
2700/2700 [==============================] - 0s 136us/step - loss: 1753.0443 - mean_squared_logarithmic_error: 4.6331
Epoch 2/50
2700/2700 [==============================] - 0s 57us/step - loss: 898.0521 - mean_squared_logarithmic_error: 3.4088
Epoch 3/50
2700/2700 [==============================] - 0s 54us/step - loss: 501.3287 - mean_squared_logarithmic_error: 2.5738
Epoch 4/50
2700/2700 [==============================] - 0s 65us/step - loss: 364.9061 - mean_squared_logarithmic_error: 1.9457
Epoch 5/50
2700/2700 [==============================] - 0s 60us/step - loss: 242.0564 - mean_squared_logarithmic_error: 1.4434
Epoch 6/50
2700/2700 [==============================] - 0s 58us/step - loss: 123.5594 - mean_squared_logarithmic_error: 1.0261
E

2700/2700 [==============================] - 0s 54us/step - loss: 25.6730 - mean_squared_logarithmic_error: 0.1377
Epoch 8/50
2700/2700 [==============================] - 0s 55us/step - loss: 23.2652 - mean_squared_logarithmic_error: 0.1255
Epoch 9/50
2700/2700 [==============================] - 0s 55us/step - loss: 20.7804 - mean_squared_logarithmic_error: 0.1160
Epoch 10/50
2700/2700 [==============================] - 0s 54us/step - loss: 19.5818 - mean_squared_logarithmic_error: 0.1115
Epoch 11/50
2700/2700 [==============================] - 0s 55us/step - loss: 19.6118 - mean_squared_logarithmic_error: 0.1109
Epoch 12/50
2700/2700 [==============================] - 0s 55us/step - loss: 18.6548 - mean_squared_logarithmic_error: 0.1086
Epoch 13/50
2700/2700 [==============================] - 0s 55us/step - loss: 17.6055 - mean_squared_logarithmic_error: 0.1026
Epoch 14/50
2700/2700 [==============================] - 0s 56us/step - loss: 16.4550 - mean_squared_logarithmic_error: 0.099

2700/2700 [==============================] - 0s 56us/step - loss: 11.4104 - mean_squared_logarithmic_error: 0.0794
Epoch 22/50
2700/2700 [==============================] - 0s 55us/step - loss: 11.4088 - mean_squared_logarithmic_error: 0.0801
Epoch 23/50
2700/2700 [==============================] - 0s 56us/step - loss: 11.1808 - mean_squared_logarithmic_error: 0.0783
Epoch 24/50
2700/2700 [==============================] - 0s 55us/step - loss: 10.7352 - mean_squared_logarithmic_error: 0.0769
Epoch 25/50
2700/2700 [==============================] - 0s 55us/step - loss: 11.3511 - mean_squared_logarithmic_error: 0.0796
Epoch 26/50
2700/2700 [==============================] - 0s 55us/step - loss: 10.8073 - mean_squared_logarithmic_error: 0.0773
Epoch 27/50
2700/2700 [==============================] - 0s 55us/step - loss: 10.7527 - mean_squared_logarithmic_error: 0.0770
Epoch 28/50
2700/2700 [==============================] - 0s 54us/step - loss: 10.6522 - mean_squared_logarithmic_error: 0.0

2700/2700 [==============================] - 0s 56us/step - loss: 11.3647 - mean_squared_logarithmic_error: 0.0798
Epoch 36/50
2700/2700 [==============================] - 0s 55us/step - loss: 10.5353 - mean_squared_logarithmic_error: 0.0759
Epoch 37/50
2700/2700 [==============================] - 0s 56us/step - loss: 10.5050 - mean_squared_logarithmic_error: 0.0766
Epoch 38/50
2700/2700 [==============================] - 0s 55us/step - loss: 10.5337 - mean_squared_logarithmic_error: 0.0762
Epoch 39/50
2700/2700 [==============================] - 0s 56us/step - loss: 10.4861 - mean_squared_logarithmic_error: 0.0765
Epoch 40/50
2700/2700 [==============================] - 0s 57us/step - loss: 10.2577 - mean_squared_logarithmic_error: 0.0753
Epoch 41/50
2700/2700 [==============================] - 0s 55us/step - loss: 10.3803 - mean_squared_logarithmic_error: 0.0758
Epoch 42/50
2700/2700 [==============================] - 0s 56us/step - loss: 10.2316 - mean_squared_logarithmic_error: 0.0

2700/2700 [==============================] - 0s 57us/step - loss: 23.7996 - mean_squared_logarithmic_error: 0.1311
Epoch 50/50
2700/2700 [==============================] - 0s 51us/step
Epoch 1/50
2700/2700 [==============================] - 1s 193us/step - loss: 739.9909 - mean_squared_logarithmic_error: 4.5410
Epoch 2/50
2700/2700 [==============================] - 0s 56us/step - loss: 264.2335 - mean_squared_logarithmic_error: 4.3357
Epoch 3/50
2700/2700 [==============================] - 0s 56us/step - loss: 218.5748 - mean_squared_logarithmic_error: 4.1006
Epoch 4/50
2700/2700 [==============================] - 0s 56us/step - loss: 181.0250 - mean_squared_logarithmic_error: 2.8907
Epoch 5/50
2700/2700 [==============================] - 0s 55us/step - loss: 155.6794 - mean_squared_logarithmic_error: 2.3336
Epoch 6/50
2700/2700 [==============================] - 0s 56us/step - loss: 144.9239 - mean_squared_logarithmic_error: 2.0602
Epoch 7/50
2700/2700 [==============================

2700/2700 [==============================] - 0s 57us/step - loss: 31.9824 - mean_squared_logarithmic_error: 0.1797
Epoch 13/50
2700/2700 [==============================] - 0s 57us/step - loss: 29.3478 - mean_squared_logarithmic_error: 0.1663
Epoch 14/50
2700/2700 [==============================] - 0s 56us/step - loss: 28.2830 - mean_squared_logarithmic_error: 0.1560
Epoch 15/50
2700/2700 [==============================] - 0s 57us/step - loss: 25.6555 - mean_squared_logarithmic_error: 0.1459
Epoch 16/50
2700/2700 [==============================] - 0s 56us/step - loss: 24.7825 - mean_squared_logarithmic_error: 0.1353
Epoch 17/50
2700/2700 [==============================] - 0s 56us/step - loss: 24.1157 - mean_squared_logarithmic_error: 0.1327
Epoch 18/50
2700/2700 [==============================] - 0s 57us/step - loss: 21.2202 - mean_squared_logarithmic_error: 0.1169
Epoch 19/50
2700/2700 [==============================] - 0s 57us/step - loss: 21.2386 - mean_squared_logarithmic_error: 0.1